In [1]:
from carsus.io.cmfgen import CMFGENReader, CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser

levels_path = "al2_osc_split.dat"  
lines_path = "al2_osc_split.dat"    

al2_lvl_parser = CMFGENEnergyLevelsParser(levels_path)
al2_osc_parser = CMFGENOscillatorStrengthsParser(lines_path)

al2_lvl_data = {
    (13, 2): {
        "levels": al2_lvl_parser.base,  # Energy levels
        "lines": al2_osc_parser.base    # Spectral lines (oscillator strengths)
    }
}

reader = CMFGENReader(al2_lvl_data)
# Print metadata from levels
print(reader.levels.attrs)

# Print metadata from lines
print(reader.lines.attrs)
print(reader.levels)  # Print levels DataFrame
print(reader.lines)  






[                py.warnings][WARNING] - /Users/pankajdhyani/Documents/GitHub/carsus/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
 (warnings.py:109)
[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for Al 2. (base.py:875)
{'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}
{'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}
                                             energy       j       label  \
atomic_number ion_charge level_index                                      
13            2          0                 0.000000     0.0   3s

### Save DataFrames with Metadata to HDF5

In [2]:
import pandas as pd

# Assume self.levels and self.lines contain metadata in .attrs
hdf_filename = "carsus_output.h5"

with pd.HDFStore(hdf_filename, "w") as store:
    store.put("levels", reader.levels)
    store.put("lines", reader.lines)

    # Attach metadata to HDF5 store
    store.get_storer("levels").attrs.metadata = reader.levels.attrs
    store.get_storer("lines").attrs.metadata = reader.lines.attrs

print(f"Saved Carsus output to {hdf_filename}")

Saved Carsus output to carsus_output.h5


In [3]:
import pandas as pd

hdf_filename = "carsus_output.h5"

with pd.HDFStore(hdf_filename, "r") as store:
    levels = store["levels"]
    lines = store["lines"]

    levels_metadata = store.get_storer("levels").attrs.metadata
    lines_metadata = store.get_storer("lines").attrs.metadata

print("Loaded Levels Metadata:", levels_metadata)
print("Loaded Lines Metadata:", lines_metadata)


Loaded Levels Metadata: {'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}
Loaded Lines Metadata: {'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}


In [5]:
def save_with_metadata(filename, df_dict):
    """
    Saves multiple Pandas DataFrames with metadata to an HDF5 file.

    Parameters:
        filename (str): Name of the HDF5 file.
        df_dict (dict): Dictionary of {name: dataframe} pairs, where each dataframe has .attrs metadata.
    """
    with pd.HDFStore(filename, "w") as store:
        for name, df in df_dict.items():
            store.put(name, df)
            store.get_storer(name).attrs.metadata = df.attrs

def load_with_metadata(filename):
    """
    Loads DataFrames and their metadata from an HDF5 file.

    Parameters:
        filename (str): Name of the HDF5 file.

    Returns:
        dict: Dictionary of {name: (dataframe, metadata)} pairs.
    """
    data = {}
    with pd.HDFStore(filename, "r") as store:
        for key in store.keys():
            df = store[key]
            metadata = store.get_storer(key.lstrip("/")).attrs.metadata
            data[key.lstrip("/")] = (df, metadata)
    return data

# Example usage:
save_with_metadata("carsus_output.h5", {"levels": reader.levels, "lines": reader.lines})

loaded_data = load_with_metadata("carsus_output.h5")
print("Loaded Metadata for Levels:", loaded_data["levels"][1])  # Check metadata
print("Loaded Metadata for Lines:", loaded_data["lines"][1])  # Check metadata


Loaded Metadata for Levels: {'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}
Loaded Metadata for Lines: {'doi': 'https://doi.org/10.1086/177435', 'reference': 'Verner et al. 1996', 'units': {'energy': 'erg', 'energy_lower': 'erg', 'energy_upper': 'erg', 'wavelength': 'meter'}}
